In [1]:
import ast
import json
import pandas as pd
import numpy as np

In [62]:
# ListResults

In [2]:
df_list = pd.DataFrame()
dist_map = {'CH': '2. Capitol Hill', 'QA': '3. Queen Anne', 'SLU': '0. SLU', 'UD': '1. University District', 'WF': '5. Wallingford & Fremont', 'DT':'4. Downtown'}
for region in ['CH', 'QA', 'SLU', 'UD', 'WF', 'DT']: 
    print(region)
    df_temp = pd.read_csv(f'data/{region}_listResults.csv')
    df_temp = df_temp[~df_temp.duplicated()]
    print(df_temp.shape)
    df_temp['Dist'] = dist_map[region]
    df_list = pd.concat([df_list, df_temp], axis = 0)


CH
(315, 56)
QA
(333, 56)
SLU
(252, 54)
UD
(299, 56)
WF
(283, 54)
DT
(306, 56)


In [3]:
df_list.columns

Index(['zpid', 'id', 'providerListingId', 'imgSrc', 'hasImage', 'detailUrl',
       'statusType', 'statusText', 'address', 'addressStreet', 'addressCity',
       'addressState', 'addressZipcode', 'units', 'lotId', 'latLong',
       'variableData', 'badgeInfo', 'isSaved', 'buildingName', 'isBuilding',
       'canSaveBuilding', 'has3DModel', 'isFeaturedListing',
       'isShowcaseListing', 'list', 'relaxed', 'carouselPhotos',
       'availabilityCount', 'marketingTreatments', 'isInstantTourEnabled',
       'isContactable', 'rawHomeStatusCd', 'marketingStatusSimplifiedCd',
       'countryCurrency', 'price', 'unformattedPrice', 'isUndisclosedAddress',
       'beds', 'baths', 'area', 'isZillowOwned', 'hdpData',
       'isUserClaimingOwner', 'isUserConfirmedClaim', 'pgapt', 'sgapt',
       'shouldShowZestimateAsPrice', 'hasVideo', 'isHomeRec',
       'hasAdditionalAttributions', 'availabilityDate', 'rooms', 'zestimate',
       'streetViewURL', 'streetViewMetadataURL', 'Dist'],
      dtype='o

In [4]:
df_all = df_list[df_list['units'].isnull()]
df_1p_unit = df_list[df_list['units'].notna()]
df_1p_unit['unitlist'] = list(map(lambda x : [] if x is np.nan else ast.literal_eval(x), df_1p_unit['units']))
max_units = max(map(len, df_1p_unit['unitlist']))
for i in range(1, max_units+1):
    filter = []
    df_temp = df_1p_unit[list(map(lambda x : len(x) == i, df_1p_unit['unitlist']))]
    count = i
    while count > 0:
        df_temp['beds'] = list(map(lambda x : x[count-1]['beds'], df_temp['unitlist']))
        df_temp['price'] = list(map(lambda x : x[count-1]['price'], df_temp['unitlist']))
        df_all = pd.concat([df_all, df_temp], axis=0)
        count -= 1
df_all = df_all.drop('unitlist', axis=1)

/var/folders/1s/89d5t8ln1xd3m_vytz6l1v2w0000gn/T/ipykernel_23074/355105244.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1p_unit['unitlist'] = list(map(lambda x : [] if x is np.nan else ast.literal_eval(x), df_1p_unit['units']))
/var/folders/1s/89d5t8ln1xd3m_vytz6l1v2w0000gn/T/ipykernel_23074/355105244.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['beds'] = list(map(lambda x : x[count-1]['beds'], df_temp['unitlist']))
/var/folders/1s/89d5t8ln1xd3m_vytz6l1v2w0000gn/T/ipykernel_23074/3

In [5]:
df_all.groupby('Dist', dropna=False)['statusType'].count()

Dist
0. SLU                      300
1. University District      347
2. Capitol Hill             365
3. Queen Anne               397
4. Downtown                 447
5. Wallingford & Fremont    295
Name: statusType, dtype: int64

In [8]:
df_all = df_all.sort_values('Dist').reset_index(drop=True)
df_all = df_all[~df_all[['statusType', 'statusText', 'address', 'addressStreet', 
                'addressCity', 'latLong', 'price', 'unformattedPrice', 'beds', 
                'baths', 'area']].duplicated()]
df_all = df_all[df_all['latLong'].apply(lambda x : len(x) > 2)]
df_all.groupby('Dist', dropna=False)['statusType'].count()

Dist
0. SLU                      210
1. University District      283
2. Capitol Hill             210
3. Queen Anne               294
4. Downtown                 306
5. Wallingford & Fremont    112
Name: statusType, dtype: int64

In [9]:
df_all['price'] = df_all['price'].str.replace('/mo', '').str.replace('$', '').str.replace('+', '').str.replace(',', '').astype(int)

In [10]:
df_all = df_all[['Dist', 'providerListingId', 'imgSrc', 'hasImage', 'detailUrl',
       'statusType', 'statusText', 'address', 'addressStreet', 'addressCity',
       'addressState', 'addressZipcode', 'units', 'latLong',
       'buildingName', 'isBuilding',
       'canSaveBuilding', 'isFeaturedListing',
       'isShowcaseListing',
       'availabilityCount', 'isInstantTourEnabled',
       'isContactable', 'rawHomeStatusCd',
       'countryCurrency', 'price', 'unformattedPrice', 'isUndisclosedAddress',
       'beds', 'baths', 'area', 'isZillowOwned', 'hdpData',
       'isUserClaimingOwner', 'isUserConfirmedClaim', 'pgapt', 'sgapt',
       'shouldShowZestimateAsPrice', 'hasVideo', 'isHomeRec',
       'hasAdditionalAttributions', 'availabilityDate', 'zestimate']]

In [11]:
df_all['latitude'] = list(map(lambda x : ast.literal_eval(x)['latitude'], df_all['latLong']))
df_all['longitude'] = list(map(lambda x : ast.literal_eval(x)['longitude'], df_all['latLong']))

In [12]:
df_all.to_csv('data/df_all_listResults.csv', index=False)

In [13]:
df_all.shape

(1415, 44)

In [14]:
# MapResults

In [18]:
df_map = pd.DataFrame()
dist_map = {'CH': '2. Capitol Hill', 'QA': '3. Queen Anne', 'SLU': '0. SLU', 'UD': '1. University District', 'WF': '5. Wallingford & Fremont', 'DT':'4. Downtown'}
for region in ['CH', 'QA', 'SLU', 'UD', 'WF', 'DT']: 
    df_temp = pd.read_csv(f'data/{region}_mapResults.csv')
    df_temp = df_temp[~df_temp.duplicated()]
    df_temp['Dist'] = dist_map[region]
    df_map = pd.concat([df_map, df_temp], axis = 0)

In [19]:
print(df_map.shape)
df_map.columns

(1407, 50)


Index(['plid', 'imgSrc', 'hasImage', 'detailUrl', 'statusType', 'statusText',
       'price', 'address', 'minBeds', 'minBaths', 'minArea', 'lotId',
       'latLong', 'variableData', 'badgeInfo', 'buildingName', 'buildingId',
       'isBuilding', 'canSaveBuilding', 'has3DModel', 'isHomeRec',
       'hasAdditionalAttributions', 'isFeaturedListing', 'isShowcaseListing',
       'listingType', 'isFavorite', 'timeOnZillow', 'unitCount',
       'rentalMarketingSubType', 'marketingTreatments', 'zpid',
       'rawHomeStatusCd', 'marketingStatusSimplifiedCd', 'priceLabel', 'beds',
       'baths', 'area', 'hdpData', 'isUserClaimingOwner',
       'isUserConfirmedClaim', 'pgapt', 'sgapt', 'shouldShowZestimateAsPrice',
       'hasVideo', 'availabilityDate', 'visited', 'info3String',
       'streetViewURL', 'streetViewMetadataURL', 'Dist'],
      dtype='object')

In [20]:
df_map['price'] = df_map['price'].str.replace('/mo', '').str.replace('$', '').str.replace('+', '').str.replace(',', '').astype(int)

In [21]:
df_map = df_map.sort_values('Dist').reset_index(drop=True)
df_map = df_map[~df_map[['price', 'address', 'minBeds', 'minBaths', 'minArea', 'lotId',
       'latLong','buildingName', 'buildingId',
       'isBuilding', 'priceLabel', 'beds',
       'baths', 'area', ]].duplicated()]

In [22]:
df_map['latitude'] = list(map(lambda x : ast.literal_eval(x)['latitude'], df_map['latLong']))
df_map['longitude'] = list(map(lambda x : ast.literal_eval(x)['longitude'], df_map['latLong']))

In [23]:
df_map.groupby(['Dist'], dropna=False)['longitude'].count()

Dist
0. SLU                      162
1. University District      235
2. Capitol Hill             175
3. Queen Anne               248
4. Downtown                 191
5. Wallingford & Fremont    105
Name: longitude, dtype: int64

In [24]:
df_map.to_csv('data/df_all_mapResult.csv', index=False)